In [ ]:
#k fold validation
import numpy as np
from numpy.linalg import inv
from sklearn.datasets import load_iris
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

iris_dataset = load_iris()
X = np.array(iris_dataset['data'])
y = np.array(iris_dataset['target'])
## one hot encoding
Y = []
for i in y:
    letter = [0,0,0]
    letter[i] = 1
    Y.append(letter)
Y = np.array(Y)
# randomize order of Y
test_idx = np.random.RandomState(seed=2).permutation(Y.shape[0])
X_test = X[test_idx[:25]]
Y_test = Y[test_idx[:25]]


error_rate_train_arr = []
error_rate_val_arr = []
## for each polynomial order find error rate of training and validation set
for order in range(1,11):
    error_rate_train_arr_fold = []
    error_rate_val_arr_fold = []
    Idx = np.random.RandomState(seed=8).permutation(Y.shape[0])
    # k fold for k=5 within order
    for k in range(0,5):
        ##polynomial classification to calculate error rate
        X_val = X[Idx[k*25:(k+1)*25]] #prepare index for fold
        Y_val = Y[Idx[k*25:(k+1)*25]]
        Idxtrain = np.setdiff1d(Idx,Idx[k*25:(k+1)*25])
        X_train = X[Idxtrain]
        Y_train = Y[Idxtrain]
        poly = PolynomialFeatures(order)
        P = poly.fit_transform(X_train)
        Pval = poly.fit_transform(X_val)
        #dual or primal form
        if P.shape[0] > P.shape[1]:
            wp = inv(P.T @ P) @ P.T @ Y_train
        else:
            inv_PTP = inv(P.T @ P)
            pinv_L = inv_PTP @ P.T
            wp = pinv_L @ Y_train
        y_est_p = P @ wp
        y_cls_p = [[1 if y==max(x) else 0 for y in x] for x in y_est_p]
        # accuracy calculation for training set
        m1tr = np.matrix(Y_train)
        m2tr = np.matrix(y_cls_p)
        diff = np.abs(m1tr - m2tr)
        error_train = np.where(diff.any(axis=1))[0]
        error_rate_train = len(error_train)/len(diff)
        error_rate_train_arr_fold += [error_rate_train]
        # accuracy calculation for validation set
        yval_est_p = Pval @ wp
        yval_cls_p = [[1 if y == max(x) else 0 for y in x] for x in yval_est_p]
        m1 = np.matrix(Y_val)
        m2 = np.matrix(yval_cls_p)
        diff = np.abs(m1-m2)
        error_val = np.where(diff.any(axis=1))[0]
        error_rate_val = len(error_val)/len(diff)
        error_rate_val_arr_fold += [error_rate_val]
    error_rate_train_arr += [np.mean(error_rate_train_arr_fold)]
    error_rate_val_arr += [np.mean(error_rate_val_arr_fold)]
